In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import TensorDataset, DataLoader
import json
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm

<h3><i>Download images</i></h3>

In [2]:
df = pd.read_csv('pizza_data_final.csv', index_col='id')

C:\Users\DuyAnh\AppData\Local\Temp\ipykernel_17136\1234846730.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pizza_data_final.csv', index_col='id')


In [3]:
all_error_list = ['Bánh không tròn | Distorted shape',
 'Cháy | Baking - Burnt',
 'Viền k đều | Edge - Uneven',
 'Bánh bé | Size - Too small',
 'Thiếu bóng | too few balls',
 'Không đốm | Baking - Does not have leopard-spotting',
 'Màu nhạt | Baking - Pale',
 'Viền nhỏ | Edge - Too small',
 'Viền to | Edge - Too big',
 'Nở viền không đủ | edge pizza is not enough swollen',
 'Viền thấp | Edge - Too low',
 'Phô mai cao | Topping - Cheese too high * with a core*',
 'Topping - Không đúng | incorrect portioning',
 'Không cân Topping - Not even half and half',
 'Sốt trên mép | Topping - Sauce covering on the edge',
 'Lên men thiếu | Fermentation - Lack fermentation',
 'Quá theo viền | Topping - Too strong shaping the edge',
 'Trộn lẫn | Topping - Bended',
 'Quá tập trung | Topping - Topping too centered',
 'Topping - Không đều | Not even',
 'Lên men quá nhiều | Fermentation - Over fermentation',
 'Không hình tròn | Topping - Not circled',
 'Bánh lớn | Size - Too big']

In [4]:
queue_download_image = []
for error in all_error_list:
    error_df = df[df[error] == 1].head(200)
    for image_id, image_url in zip(error_df.index, error_df['image_url']):
        queue_download_image.append((image_id, image_url))

In [5]:
# data_path = 'data/pizza_image'
# downloaded = []
# for id, url in tqdm(queue_download_image):
#     if id not in downloaded:
#         downloaded.append(id)
#         response = requests.get(url)
#         image_data = BytesIO(response.content)
#         image = Image.open(image_data)
#         image_file_name = os.path.join(data_path, f"{id}.jpg")
#         image.save(image_file_name)
#         if response.status_code == 200:
#             pass
#         else:
#             print(f"Failed to download {image_url}")

In [6]:
images = {}
dict_labels = {}
for id, _ in tqdm(queue_download_image):
    images[id] = np.array(Image.open(f'data/pizza_image/{id}.jpg'))
    dict_labels[id] = []
    for error in all_error_list:
        dict_labels[id].append(df.at[id, error])

  0%|          | 0/4126 [00:00<?, ?it/s]

100%|██████████| 4126/4126 [01:33<00:00, 44.01it/s]


In [7]:
print(images['644b407dba8c270029f69526'], dict_labels['644b407dba8c270029f69526'])

[[[ 0 11  5]
  [ 4 15  7]
  [ 8 19 11]
  ...
  [ 0  4 15]
  [ 0  4 13]
  [ 0  4 13]]

 [[ 8 17 12]
  [ 9 20 14]
  [11 21 13]
  ...
  [ 0  3 12]
  [ 0  3 12]
  [ 0  3 10]]

 [[12 18 14]
  [10 19 14]
  [11 18 11]
  ...
  [ 0  0 10]
  [ 0  0  8]
  [ 0  1  6]]

 ...

 [[39 43 55]
  [33 35 50]
  [31 30 48]
  ...
  [13  7 43]
  [10  4 42]
  [12  8 45]]

 [[41 45 56]
  [43 45 57]
  [45 44 58]
  ...
  [17 13 50]
  [13  9 44]
  [19 18 52]]

 [[38 42 51]
  [47 50 59]
  [52 51 65]
  ...
  [11  9 46]
  [ 6  5 39]
  [17 16 50]]] [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
model = torchvision.models.resnet50(pretrained = True)
num_classes = 23
in_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(in_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256, num_classes),
    nn.Sigmoid()
)

In [10]:
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

In [11]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def transform_image(image):
    image = Image.fromarray(image)
    img_tensor = transform(image)
    return img_tensor

In [12]:
new_images = {}
for id, img in tqdm(images.items()):
    new_images[id] = transform_image(img) 

100%|██████████| 2416/2416 [01:48<00:00, 22.19it/s]


In [13]:
new_images['644b407dba8c270029f69526'].shape

torch.Size([3, 224, 224])

In [14]:
new_labels = []
for id, label in dict_labels.items():
    new_labels.append(label)

new_labels = torch.tensor(new_labels, dtype = torch.float)
new_labels

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 1., 0.,  ..., 0., 0., 1.],
        [0., 1., 0.,  ..., 0., 0., 1.]])

In [15]:
images.clear()

In [16]:
from sklearn.model_selection import train_test_split
batch_size = 32
X_train, X_test, y_train, y_test = train_test_split(list(new_images.values()), new_labels, test_size=0.2, random_state=42)

train_data = TensorDataset(torch.stack(X_train), y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = TensorDataset(torch.stack(X_test), y_test)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [17]:
print("Kích thước của train_data:", len(train_data), train_data[0][0].shape, train_data[0][1].shape)
print("Tổng số batch:", len(train_loader))

Kích thước của train_data: 1932 torch.Size([3, 224, 224]) torch.Size([23])
Tổng số batch: 61


In [36]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện mô hình
num_epochs = 3
for epoch in tqdm(range(num_epochs)):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Đánh giá mô hình (nếu bạn có tập kiểm tra)
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predicted = torch.round(outputs)
            total += labels.size(0) * 23
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [07:15<14:31, 435.72s/it]

Epoch 1/3, Loss: 14.5528, Accuracy: 67.78%


 67%|██████▋   | 2/3 [14:13<07:04, 424.92s/it]

Epoch 2/3, Loss: 11.0600, Accuracy: 70.12%


100%|██████████| 3/3 [21:19<00:00, 426.42s/it]

Epoch 3/3, Loss: 11.3137, Accuracy: 67.94%


In [37]:
torch.save(model.state_dict(), 'model_1.pth')

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
for epoch in tqdm(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total += labels.size(0) * 
    model.eval()
    with torch.inference_mode():
        for inputs, labels in test_loader:
            outputs = model(inputs)

<h4> Predict pizza image </h4>

In [49]:
def predict(img_tensor):
    img_tensor = img_tensor.unsqueeze(dim = 0)
    predicted = model(img_tensor)
    predicted = torch.round(predicted)
    print(predicted)

In [56]:
dict_labels = {}
for id, _ in tqdm(queue_download_image):
    dict_labels[id] = []
    for error in all_error_list:
        dict_labels[id].append(df.at[id, error])

  0%|          | 0/4126 [00:00<?, ?it/s]

100%|██████████| 4126/4126 [00:01<00:00, 3431.51it/s]


In [58]:
my_image_path = 'data/pizza_image/644b4f88ba8c270029f69584.jpg'
my_image = np.array(Image.open(my_image_path))
my_image_tensor = transform_image(my_image)
predict(my_image_tensor)
print(dict_labels['644b4f88ba8c270029f69584'])

tensor([[1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]], grad_fn=<RoundBackward0>)
[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
